In [3]:
# Gerekli modüllerin içe aktarılması
import sys
import os
import pandas as pd
from sqlalchemy import create_engine

# Proje dizininin sys.path'e eklenmesi (eğer gerekliyse)
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(project_root)

# config.py dosyasının içe aktarılması
from config import config

# Veritabanı bağlantısının kurulması
db_config = config.DATABASE
connection_string = f"{db_config['drivername']}://" \
                    f"{db_config['username']}:{db_config['password']}@" \
                    f"{db_config['host']}:{db_config['port']}/" \
                    f"{db_config['database']}"

engine = create_engine(connection_string)

# Verilerin veritabanından okunması
global_50_df = pd.read_sql('SELECT * FROM spotify_tracks', con=engine)

# Verilerin görüntülenmesi
global_50_df.head()


,track_name,artist_name,album_name,release_date,total_tracks,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
0,APT.,ROSÉ,APT.,2024-10-18,1,95,0.777,0.783,0,-4.477,0,0.2600,0.02830,0.0000,0.355,0.939,149.027,5vNRhkKd0yEAg8suGBpjeY,4
1,Die With A Smile,Lady Gaga,Die With A Smile,2024-08-16,1,100,0.521,0.592,6,-7.777,0,0.0304,0.30800,0.0000,0.122,0.535,157.969,2plbrEY59IikOBgBGLjaoe,3
2,BIRDS OF A FEATHER,Billie Eilish,HIT ME HARD AND SOFT,2024-05-17,10,97,0.747,0.507,2,-10.171,1,0.0358,0.20000,0.0608,0.117,0.438,104.978,6dOtVTDdiauQNBQEDOtlAB,4
3,St. Chroma (feat. Daniel Caesar),"Tyler, The Creator",CHROMAKOPIA,2024-10-28,14,83,0.576,0.884,0,-5.058,1,0.5550,0.35700,0.0000,0.368,0.373,150.450,1QoyuMHNBe7lg3YW4Qtll4,4
4,Who,Jimin,MUSE,2024-07-19,7,92,0.660,0.756,0,-3.743,0,0.0320,0.00289,0.0000,0.193,0.838,116.034,7tI8dRuH2Yc6RuoTjxo4dU,4


In [18]:
#ses ozellikleri
# acousticness -> bir parçanın akustik olup olmadığının ölçüsü 0.0 ila 1.0 arasi bir deger ile belirler. 
# danceability -> bir parçanın dans edilebilirliğini belirleyen bir değerdir. Tempo, rhythm stabilty, beat strength, overall regularity gibi müzikal değelere göre belirlernir. 0.0 - 1.0 arası değer alır 
# energy ->  bir parçanın yoğunluk aktivite düzeyine ölçer. Yüksek enerji, hızlı, gürültülü, dinamik parçaları temsil eder. 0.0 - 1.0
# Instrumentalness -> Bir parçada vokal içerik olmama olasılığını tahmin eder. 0.0 - 1.0
# Liveness -> Bir parçanın canlı bir performanstan kaydedilip kaydedilmediğini algılar. 0.0 - 1.0
# Speechiness -> Bir parçadaki konuşma içeriğinin varlığını algılar. 0.0 - 1.0
# Valence -> Bir parçanın müzikal pozitifliğinin veya mutluluğunun ölçüsüdür. 0.0 - 1.0
# Tempo -> Bir parçanın dakikadaki vuruş sayısı (BPM) cinsinden tahmin edilen temposudur.
# Time Signature -> Bir ölçüdeki vuruş sayısını belirten bir işarettir.
# Key (Ton Anahtarı) -> Bir parçanın tahmini ton anahtarıdır. 0 - 11\
# Mode -> Bir parçanın modunu belirtir (majör veya minör). Değer 0 ise minör, 1 ise majör modu temsil eder.
# Loudness -> Bir parçanın desibel (dB) cinsinden genel ses yüksekliği. Ses yüksekliği değerlerinin tüm parça boyunca ortalaması alınır ve parçaların göreceli ses yüksekliğini karşılaştırmak için kullanışlıdır. Yükseklik, fiziksel gücün (genlik) birincil psikolojik karşılığı olan bir sesin kalitesidir. Değerler tipik olarak -60 ile 0 db arasında değişir.
global_50_df.tail()

,track_name,artist_name,album_name,release_date,total_tracks,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
45,Too Sweet,Hozier,Unreal Unearth: Unaired,2024-08-19,23,84,0.740,0.643,10,-5.446,1,0.0413,0.0410,0.00229,0.0407,0.960,117.025,19XpFsce28aByvCC4g89tJ,4
46,The Emptiness Machine,Linkin Park,The Emptiness Machine,2024-09-05,1,91,0.466,0.872,7,-3.344,1,0.0336,0.0156,0.00000,0.1210,0.806,184.115,2PnlsTsOTLE5jnBnNe2K0A,4
47,I'll Be There,Jin,I'll Be There,2024-10-25,1,82,0.538,0.738,10,-2.412,1,0.0445,0.0351,0.00000,0.0898,0.736,148.887,2Qcbspnftnon37jibwyvMU,4
48,Story of My Life,One Direction,Midnight Memories (Deluxe),2013-11-25,18,87,0.600,0.663,3,-5.802,1,0.0477,0.2250,0.00000,0.1190,0.286,121.070,4nVBt6MZDDP6tRVdQTgxJg,4
49,DOS DÍAS,Tito Double P,INCÓMODO,2024-08-22,21,88,0.745,0.513,1,-9.116,1,0.0870,0.7830,0.00000,0.2160,0.929,109.935,5lKFO0PrOC9jmOlPEdjHsL,3
